In [ ]:
# !sudo apt-get install xvfb
# !pip install bs4 selenium pyvirtualdisplay PyPDF2 requests pillow
# !pip install playwright bs4 pillow requests pypdf2
# !playwright install
# !playwright install-deps

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from pyvirtualdisplay import Display
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

from PIL import Image
import requests
import time
from io import BytesIO
from PyPDF2 import PdfMerger
import os

In [2]:
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# Replace with your remote Selenium server URL
driver = webdriver.Chrome(
    # command_executor='https://hoa-reformed-kimberlee.ngrok-free.dev',
    # options=options
)

In [3]:
url = "https://weebcentral.com/series/01J76XYCERXE60T7FKXVCCAQ0H/Jujutsu-Kaisen"

In [4]:
driver.get(url)
button = driver.find_element(By.CSS_SELECTOR, 'button[hx-target="#chapter-list"]')
button.click()

time.sleep(5)

In [6]:
html_content = driver.page_source
soup = BeautifulSoup(html_content, 'html.parser')

In [7]:
chapters = soup.find(id="chapter-list").find_all("a")
links = [chapter["href"] for chapter in chapters]
links.reverse()

In [42]:
start_chapter = 255
num_chapters = 9
vol = 29

driver.get(links[start_chapter])
required = links[start_chapter:start_chapter+num_chapters]

In [30]:
required

['https://weebcentral.com/chapters/01J76XZ7KQ224GES0JKSDQF2QG',
 'https://weebcentral.com/chapters/01J76XZ7MMS0QQHTZG7HY7546F',
 'https://weebcentral.com/chapters/01J76XZ7YEYS7KHWCWP0PD5VP6',
 'https://weebcentral.com/chapters/01J76XZ7Z7RCNSB050DE8E5V14',
 'https://weebcentral.com/chapters/01J76XZ7ZZBSTPMNGBKHSQRX9D',
 'https://weebcentral.com/chapters/01J76XZ802CTS5APT59NFW1HFT',
 'https://weebcentral.com/chapters/01J76XZ80H587JZF0YTXCT6RS3',
 'https://weebcentral.com/chapters/01J76XZ814RYER2PTTWKWXXNPK',
 'https://weebcentral.com/chapters/01J76XZ8AQWSP91NBR1MFEHMYR']

In [31]:
all_image_links = []

for link in required:
    driver.get(link)
    time.sleep(5)  # Wait for the page to load completely
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    
    title = soup.find("title").text.split(" | ")
    name, chapter = title[1], title[0].split()[1]

    image_section = soup.select_one('section[hx-include="[name=\'reading_style\']"]')
    image_tags = image_section.find_all("img")
    image_links = [img["src"] for img in image_tags]
    print(f"Found {len(image_links)} images for {name} - {chapter}")
    all_image_links.extend(image_links)

    continue

    images = []
    
    for img_url in image_links:
        response = requests.get(img_url)
        img = Image.open(BytesIO(response.content)).convert("RGB")
        images.append(img)
    
    if images:
        pdf_path = f"./pdfs/{name} - {chapter}.pdf"
        images[0].save(pdf_path, save_all=True, append_images=images[1:])
        print(f"Saved {pdf_path}")

Found 28 images for Jujutsu Kaisen - 246
Found 20 images for Jujutsu Kaisen - 247
Found 19 images for Jujutsu Kaisen - 248
Found 19 images for Jujutsu Kaisen - 249
Found 20 images for Jujutsu Kaisen - 250
Found 18 images for Jujutsu Kaisen - 251
Found 21 images for Jujutsu Kaisen - 252
Found 20 images for Jujutsu Kaisen - 253
Found 23 images for Jujutsu Kaisen - 254


In [34]:
title = f"Jujutsu Kaisen - Vol {vol}"
json_payload = {
    "folder": f"vol-{vol}",
    "image_links": all_image_links,
}

In [35]:
len(all_image_links)

188

In [37]:
res = requests.post("http://localhost:8080/download", json=json_payload)
print(res.status_code)

202


In [45]:
driver.quit()

In [ ]:
import os
from PIL import Image

vol = 26

folder = f"D:\\repository\\otaku\\vol-{vol}"  # Change to your folder name
images = []

# Collect image paths sorted by index
image_files = sorted(
    [f for f in os.listdir(folder) if f.endswith(".jpg")],
    key=lambda x: int(os.path.splitext(x)[0])
)

for filename in image_files:
    img_path = os.path.join(folder, filename)
    img = Image.open(img_path).convert("RGB")
    images.append(img)

if images:
    images[0].save(
        f"{folder}.pdf",
        save_all=True,
        append_images=images[1:]
    )